In [1]:
import pandas as pd 
import numpy as np

## 1. Carga fichero.

In [2]:
df = pd.read_csv("../Data/train.csv")
df.head()

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size,salario_eur
0,2022,140806.99,2,33,26,0,25,2,33831.16
1,2023,96163.86,2,89,75,100,70,2,139656.00
2,2022,140806.99,2,25,75,0,70,2,92000.00
3,2022,140806.99,2,33,75,100,70,2,144072.00
4,2023,140806.99,2,33,75,100,70,2,161000.00


In [3]:
df.describe()

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size,salario_eur
count,3004.000000,3004.000000,3004.000000,3004.000000,3004.000000,3004.000000,3004.000000,3004.000000,3004.000000
mean,2022.367177,126571.203648,1.995672,39.836551,66.347204,46.970706,62.309920,2.083888,127011.510200
std,0.691994,25848.612573,0.142458,18.529065,19.071250,48.632424,17.306587,0.390897,58054.043449
min,2020.000000,72262.580000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,4721.440000
25%,2022.000000,96163.860000,2.000000,29.000000,75.000000,0.000000,70.000000,2.000000,87400.000000
50%,2022.000000,140806.990000,2.000000,33.000000,75.000000,0.000000,70.000000,2.000000,124200.000000
75%,2023.000000,140806.990000,2.000000,47.000000,75.000000,100.000000,70.000000,2.000000,162150.000000
max,2023.000000,179336.460000,3.000000,92.000000,77.000000,100.000000,71.000000,3.000000,414000.000000


## 2. Creacion X/Y.

In [4]:
from sklearn.model_selection import train_test_split

In [6]:
X = df.drop(columns=["salario_eur"], axis=1)
y = df["salario_eur"]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(600, 8)
(2404, 8)
(600,)
(2404,)


### 3. Modelo baseline.

In [8]:
from sklearn.linear_model import LinearRegression

In [9]:
baseline_model = LinearRegression()
baseline_model.fit(X_train, y_train)

LinearRegression()

In [10]:
baseline_predictions = baseline_model.predict(X_test)

In [11]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score

In [13]:
print("MAE", mean_absolute_error(y_test, baseline_predictions))
print("MSE", mean_squared_error(y_test, baseline_predictions))
print("RMSE", np.sqrt(mean_squared_error(y_test, baseline_predictions)))
print("accuracy_score", accuracy_score(y_test, baseline_predictions))

MAE 37187.196211039125
MSE 2484636558.0268383
RMSE 49846.12881685837


ValueError: continuous is not supported

#### 4. Polynomial Regression

In [36]:
from sklearn.preprocessing import PolynomialFeatures

poly_reg = PolynomialFeatures(degree=5)
poly_reg.fit(X_train)
X_train_poly = poly_reg.transform(X_train)
X_test_poly = poly_reg.transform(X_test)

pol_reg_2 = LinearRegression()
pol_reg_2.fit(X_train_poly, y_train)

LinearRegression()

In [46]:
train_prediction = pol_reg_2.predict(X_train_poly)

print("scoore" , pol_reg_2.score(X_train_poly, y_train))

print("MAE", mean_absolute_error(y_train, train_prediction))
print("MSE", mean_squared_error(y_train, train_prediction))
print("RMSE", np.sqrt(mean_squared_error(y_test, train_prediction)))
print("accuracy_score", accuracy_score(y_test, train_prediction))


scoore 0.5579698350995633
MAE 28379.551535556566
MSE 1399265583.0923069


ValueError: Found input variables with inconsistent numbers of samples: [2404, 600]

Mejora el resultado pero sigue dando margen a mejora.

#### 4. Regression Tree

In [38]:
from sklearn.tree import DecisionTreeRegressor

In [39]:
from sklearn.model_selection import GridSearchCV

model = DecisionTreeRegressor(random_state=0)

parameters = {"max_depth":[2,3,4,5,6,7],
              "criterion": ['squared_error', 'absolute_error'],
              "min_samples_split": [2,4,6,8,10,12],
              "min_samples_leaf": [2,5,10,15],
              "max_features": [1,2]
}

dtr_gs = GridSearchCV(model, parameters, cv=5, scoring="neg_mean_absolute_error")

In [40]:
dtr_gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=0),
             param_grid={'criterion': ['squared_error', 'absolute_error'],
                         'max_depth': [2, 3, 4, 5, 6, 7],
                         'max_features': [1, 2],
                         'min_samples_leaf': [2, 5, 10, 15],
                         'min_samples_split': [2, 4, 6, 8, 10, 12]},
             scoring='neg_mean_absolute_error')

In [41]:
dtr_gs.best_params_

{'criterion': 'squared_error',
 'max_depth': 7,
 'max_features': 2,
 'min_samples_leaf': 5,
 'min_samples_split': 12}

In [42]:
y_pred_tree = dtr_gs.best_estimator_.predict(X_test)

print("R2", r2_score(y_test, y_pred_tree))
print("MAE", mean_absolute_error(y_test, y_pred_tree))
print("MAPE", mean_absolute_percentage_error(y_test, y_pred_tree))
print("MSE", mean_squared_error(y_test, y_pred_tree))
print("RMSE", np.sqrt(mean_squared_error(y_test, y_pred_tree)))

R2 0.3188061741846696
MAE 35466.816257621096
MAPE 0.3879155418091792
MSE 2328830639.2673573
RMSE 48257.95933592051


In [9]:
import pickle

with open('../models/train_model.pkl', 'wb') as archivo_salida:
    pickle.dump(baseline_model, archivo_salida)